In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

# 1. `scipy.sparse`: řídké matice

Po modulu `scipy.special` se přesuneme k řídkým maticím. Řídká matice obsahuje převážně nulové prvky, takže se vyplatí ukládat jen nenulové hodnoty a jejich pozice.

## 1.1 Nejčastější formáty

- `coo_matrix`: souřadnicový formát (řádek, sloupec, hodnota), vhodný pro sestavení matice.
- `csr_matrix`: Compressed Sparse Row, vhodný pro řádkové operace a násobení.
- `csc_matrix`: Compressed Sparse Column, vhodný pro sloupcové operace a rozklady.
- `lil_matrix`: List of Lists, vhodný pro postupné doplňování prvků.

In [ ]:
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, lil_matrix
import scipy.sparse as sparse

## 1.2 Jak vypadají data v jednotlivých formátech

Ukážeme si, jak se stejné hodnoty ukládají v různých sparse formátech.

In [ ]:
data = np.array([1, 2, 4, 5, 6])
row_indices = np.array([0, 0, 2, 2, 2])
col_indices = np.array([0, 2, 0, 1, 2])

# což odpovídá plné matici
full_matrix = np.array([[1, 0, 2],
                        [0, 0, 0],
                        [4, 5, 6]])

print("Full matrix:")
print(full_matrix)
print()

# CSR matrix
csr = csr_matrix((data, (row_indices, col_indices)))
print("CSR format:")
print("data:", csr.data)
print("indices:", csr.indices)
print("indptr:", csr.indptr)
print()

# CSC matrix
csc = csc_matrix((data, (row_indices, col_indices)))
print("CSC format:")
print("data:", csc.data)
print("indices:", csc.indices)
print("indptr:", csc.indptr)
print()

# COO matrix
coo = coo_matrix((data, (row_indices, col_indices)))
print("COO format:")
print("data:", coo.data)
print("row:", coo.row)
print("col:", coo.col)
print()

# LIL matrix
lil = lil_matrix((3, 3))
lil[0, 0] = 1
lil[0, 2] = 2
lil[2, 0] = 4
lil[2, 1] = 5
lil[2, 2] = 6
print("LIL format:")
print("data:", lil.data)
print("rows:", lil.rows)
print()


## 1.3 Převody mezi formáty

Řídké matice lze snadno převádět mezi formáty. Pro kontrolu nebo demonstraci je můžeme převést na plnou matici pomocí `todense()`.

In [ ]:
A = np.array([[1, 0, 2], [0, 0, 0], [4, 5, 6]])

# vytvoření CSR matice z plné matice
A_csr = csr_matrix(A)
print(repr(A_csr))

# převod z CSR do LIL
A_lil = lil_matrix(A_csr)
print(repr(A_lil))

# převod na plnou matici
A_full = A_csr.todense()
print(repr(A_full))

## 1.4 Praktická doporučení

Řídkou matici je nejčastěji nejlepší sestavit ze souřadnic nenulových prvků a jejich hodnot.
Při postupném doplňování prvků používej `lil_matrix`; u ostatních formátů je tato operace výrazně pomalejší.

Při sestavování přes souřadnice může být jedna pozice uvedená vícekrát. Hodnoty se v takovém případě sečtou, což je užitečné například při skládání globální matice v metodě konečných prvků.

In [ ]:
# řídká matice z COO s více hodnotami ve stejných buňkách
indexy_radku = np.array([0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2])
indexy_sloupcu = np.array([0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2])
data = np.array([1, 2, 3, 4, 5, 6, 5, 4, 3, 2, 1, 0])

A = coo_matrix((data, (indexy_radku, indexy_sloupcu)))
print(A.todense())

## 1.5 Matice se speciální strukturou

- `sp.eye(n)` vytvoří jednotkovou matici velikosti `n × n`.
- `sp.diags(diagonals, offsets)` vytvoří matici zadanou diagonálami a jejich posuny.

# 2. `scipy.sparse.linalg`: lineární algebra pro řídké systémy

Navazující modul `scipy.sparse.linalg` nabízí přímé i iterativní řešiče a nástroje pro rozklady.

- `spsolve` - přímé řešení soustavy lineárních rovnic.
- `spsolve_triangular` - řešení pro trojúhelníkové matice.
- `cg`, `gmres`, `minres` - iterativní metody.
- `eigs`, `eigsh`, `svds` - odhad části spektra.
- `splu`, `spilu` - LU rozklady.

In [ ]:
# submoduly je vždy lepší importovat explicitně, pomůžeme tím IDE a zlepšíme čitelnost kódu
import scipy.sparse.linalg as sla

In [ ]:
# vytvoříme třídiagonální řídkou matici s 2 na hlavní diagonále a -1 na vedlejších diagonálách
n = 5
A = sparse.diags([-1, 2, -1], [-1, 0, 1], shape=(n, n), format="csc")
print(repr(A))
print(A.todense())
# a pravou stranu
b = np.ones(n)/n

## 2.1 Přímé řešení pomocí `spsolve`

`spsolve` je přímý sparse řešič. Konkrétní interní metoda se může lišit podle typu matice a sestavení SciPy.

In [ ]:
# vyřešíme soustavu přímým řešičem
x = sla.spsolve(A, b)
plt.plot(x)

In [ ]:
# sestavíme LU dekompozici a použijeme ji pro řešení soustavy

# LU dekompozice vrací speciální objekt obsahující všechny potřebné součásti dekompozice
lu_decomp = sla.splu(A)
print(lu_decomp)

In [ ]:

# obtain matrices L, U and permutation P (ordering vector)
L = lu_decomp.L
U = lu_decomp.U
P_permutace = lu_decomp.perm_c

print("Lower triangular matrix L:")
print(repr(L))
print(L.todense())
print()

print("Upper triangular matrix U:")
print(repr(U))
print(U.todense())
print()

print("Permutation matrix P:")
print(P_permutace)

# sestavíme permutační matici P z vektoru P
# syntaxe byla: csc_matrix((data, (indexy_radku, indexy_sloupcu)))
P = csc_matrix((np.ones_like(P_permutace), (P_permutace, np.arange(n))))
print(P.todense())

## 2.2 Kontrola LU rozkladu

In [ ]:
print("P^T * L * U * P:")
print((P.T @ L @ U @ P).todense())
print()
print("A:")
print(A.todense())


## 2.3 Řešení s trojúhelníkovými maticemi

`spsolve_triangular` je vhodné použít přímo pro trojúhelníkové soustavy.

In [ ]:
# při řešení s dekompozicí řešíme soustavu P^T * L * U * P * x = b
# nejdříve tedy L * y = P * b
y = sla.spsolve_triangular(L, P @ b, lower=True)

In [ ]:
# a poté soustavu U * P * x = y
x = P.T @ sla.spsolve_triangular(U, y, lower=False)

In [ ]:
plt.plot(x)

## 2.4 Řešení přes objekt LU

Objekt LU rozkladu umí soustavy s danou pravou stranou řešit přímo metodou `solve`.

In [ ]:
# řešení pomocí LU dekompozice lze udělat i jednodušeji
x = lu_decomp.solve(b)
plt.plot(x)

## 2.5 Odhad části spektra

Pomocí `eigs`/`eigsh` umíme spočítat vybraná vlastní čísla a vektory bez plného spektrálního rozkladu.

In [ ]:
# spočítáme vlastní čísla a vlastní vektory
# eigs počítá vlastní čísla a vlastní vektory pro obecnou matici
# eigsh počítá vlastní čísla a vlastní vektory pro symetrickou (Hermitovskou) matici

w, v = sla.eigsh(A, k=3)
print("Eigenvalues:")
print(w)
print()
print("Eigenvectors:")
print(v)
